# Convergence Test

In [1]:
import sys
import random
print(sys.version)

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Layout
from plotly.validator_cache import ValidatorCache
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import DeepDataset as sts
import DeepMetrics as dms
import DeepNNTorch as dnn

from importlib import reload
from tqdm.auto import tqdm
from scipy import stats
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]


# PyTorch

In [2]:
import torch
from torch import nn
from torchsummary import summary
import torch.nn.functional as F
torch.manual_seed(111) #test
random.seed(111)

# Input dataset

The function read_img takes as input the folder of a class (all files belongin to the same class), the class itself (e.g. 0, 1, ...), and the size of the image. The function will resize the image to have aspect ratio equals to one.

In [3]:
input_class0, label_class0 = sts.read_img("./datasets/class0/", 0, 80)
input_class1, label_class1 = sts.read_img("./datasets/class1/", 1, 80)

input_data = np.concatenate((input_class0, input_class1), axis=0)
input_label = np.concatenate((label_class0, label_class1), axis=0)
print("Input dataset shape", np.shape(input_data), "Labels", np.shape(input_label))

Input dataset shape (264, 1, 80, 80) Labels (264,)


## Split dataset

In [4]:
train_features, test_features, train_labels, test_labels = train_test_split(
    input_data, input_label, test_size = 0.20, random_state = 123)

print('train', train_features.shape)
print('train labels', train_labels.shape)
print('test', test_features.shape)
print('test labels', test_labels.shape)

train (211, 1, 80, 80)
train labels (211,)
test (53, 1, 80, 80)
test labels (53,)


## Dataset to Tensor

In [5]:
# input data
data_ = train_features
labels_ = train_labels
labels_test = test_labels

# input data tensor
data_tensor = torch.tensor(train_features, dtype=float)
labels_tensor = torch.tensor(train_labels, dtype=float)
print(data_tensor.shape, data_tensor[0].shape)
print(labels_tensor.shape, labels_tensor[0].shape)

# test data tensor
data_tensor_test = torch.tensor(test_features, dtype=float)
labels_tensor_test = torch.tensor(test_labels, dtype=float)
print(data_tensor_test.shape, data_tensor[0].shape)
print(labels_tensor_test.shape, labels_tensor[0].shape)

torch.Size([211, 1, 80, 80]) torch.Size([1, 80, 80])
torch.Size([211]) torch.Size([])
torch.Size([53, 1, 80, 80]) torch.Size([1, 80, 80])
torch.Size([53]) torch.Size([])


## Dataset to batches

In [6]:
# input data training -> batch
batch = 32
batch_n = int(np.ceil(data_.shape[0] / batch))

indx_str = 0
indx_end = batch

input_data_training = []
labels_training = []
for i in range(batch_n):
    input_data_batch = data_[indx_str:indx_end]
    inputs_tensor = torch.tensor(input_data_batch, dtype=float)
    input_data_training.append(inputs_tensor)
    
    labels_batch = labels_[indx_str:indx_end]
    labels_barch_tensor = torch.tensor(labels_batch, dtype=float)
    labels_training.append(labels_barch_tensor)
    
    
    indx_str += batch
    indx_end += batch

print('input data', len(input_data_training), input_data_training[0].shape)
print('labels', len(labels_training), labels_training[0].shape)

input data 7 torch.Size([32, 1, 80, 80])
labels 7 torch.Size([32])


# CNN PyTorch model

## CNN Architecture

In [7]:
conv_chanels = [1, 8, 16, 32, 64]
conv_kernel = [2, 2, 2, 2]
conv_stride = [1, 2, 1, 2]
conv_padding = [0, 0, 0, 0]
conv_dilation = [1, 1, 1, 1]
conv_out = [3,3]

pool_kernel = [2, 2, 2, 2]
pool_stride = [2, 1, 2, 1]
pool_padding = [0, 0, 0, 0]
pool_dilation = [1, 1, 1, 1]

In [8]:
dnn = reload(dnn)
torch.manual_seed(111) #test
random.seed(111)
cnnmodel = dnn.CNNt(conv_chanels, conv_kernel, conv_stride, conv_padding,
                    pool_kernel, pool_stride, conv_out)

fcn architecture [576, 128, 64, 32, 2]


## CNN Summary

In [9]:
summary(cnnmodel, input_size = (1, 80, 80))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 79, 79]              40
       BatchNorm2d-2            [-1, 8, 79, 79]              16
              ReLU-3            [-1, 8, 79, 79]               0
         MaxPool2d-4            [-1, 8, 39, 39]               0
            Conv2d-5           [-1, 16, 19, 19]             528
       BatchNorm2d-6           [-1, 16, 19, 19]              32
              ReLU-7           [-1, 16, 19, 19]               0
         MaxPool2d-8           [-1, 16, 18, 18]               0
            Conv2d-9           [-1, 32, 17, 17]           2,080
      BatchNorm2d-10           [-1, 32, 17, 17]              64
             ReLU-11           [-1, 32, 17, 17]               0
        MaxPool2d-12             [-1, 32, 8, 8]               0
           Conv2d-13             [-1, 64, 4, 4]           8,256
      BatchNorm2d-14             [-1, 6

## Loss function & Params

In [26]:
cnnmodel.train()

lr = 0.001
cnn_loss = torch.nn.CrossEntropyLoss()
cnn_smx = torch.nn.Softmax(dim=1)
cnn_optm = torch.optim.Adam(cnnmodel.parameters(), lr=lr)

## Train

In [27]:
# Weights initialization
torch.manual_seed(111) #test
random.seed(111)
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
cnnmodel = cnnmodel.apply(weights_init)

# iteration
n_epochs = 2
cnn_costs = []
for epoch in tqdm(range(n_epochs)):
    acc_loss = 0
    for indx in range(len(input_data_training)):
        
        #feedforward
        cnn_optm.zero_grad()
        outputs = cnnmodel(input_data_training[indx].float())
        
        # loss function & backprop
        loss = cnn_loss(outputs, labels_training[indx].long())
        
        loss.backward()
        cnn_optm.step()
        
        acc_loss += loss.item()
    cnn_costs.append(acc_loss/len(input_data_training))

print(f"Epoch {epoch} mean loss {acc_loss / len(input_data_training)}")

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1 mean loss 0.5999843307903835


## Cost curve

In [28]:
cnn_costs_array = np.array(cnn_costs)
x = [int(x) for x in range(len(cnn_costs_array))]
data_cost_training = {'epoch': x, 'cost': cnn_costs_array}
df_costs = pd.DataFrame(data_cost_training)
fig = px.line(df_costs, x="epoch", y="cost", line_shape="spline")
fig.show()

## Test

In [29]:
# Test
data_eval = data_tensor_test.float()
labels_eval = labels_test

print(data_eval.shape)
print(labels_eval.shape)

# classification
th = 0.5
outputs = cnnmodel(data_eval)
outputs_softmax = cnn_smx(outputs).detach().numpy()[:,1]
outputs_bi = np.zeros(outputs_softmax.shape[0])
indx_success = outputs_softmax > th
outputs_bi[indx_success] = 1

# metrics
tp, tn, fp, fn, tpr, fpr = dnn.pred_metrics(labels_eval, outputs_bi)
fpr_, tpr_, thresholds = roc_curve(labels_eval, outputs_bi, pos_label=1)
auc_ = auc(fpr_, tpr_)
f1 = round(f1_score(labels_eval, outputs_bi, average='binary'),3)

# show metrics
print('confusion matrix\n',confusion_matrix(labels_eval, outputs_bi))
print('\nf1', round(f1,3))
print('tpr', round(tpr,3))
print('fpr', round(fpr,3))
print('auc', round(auc(fpr_, tpr_),3))

torch.Size([53, 1, 80, 80])
(53,)
confusion matrix
 [[24  0]
 [ 0 29]]

f1 1.0
tpr 1.0
fpr 0.0
auc 1.0


## ROC/DET curve and Metrics

In [30]:
preds = outputs_softmax
output = np.zeros(labels_eval.shape[0])
ths = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

In [31]:
dms = reload(dms)
bpcer_th, apcer_th, auc_th, tpr_list, fpr_list = dms.pad_metrics(labels_eval, preds, ths)

N[PAIS]: 29
N[BF]: 24
0.1 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.15 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.2 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.25 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.3 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.35 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.4 auc 0.5 f1 0.707 bpcer 1.0 apcer 0.0
0.45 auc 0.979 f1 0.983 bpcer 0.042 apcer 0.0
0.5 auc 1.0 f1 1.0 bpcer 0.0 apcer 0.0
0.55 auc 1.0 f1 1.0 bpcer 0.0 apcer 0.0
0.6 auc 0.983 f1 0.982 bpcer 0.0 apcer 0.034
0.65 auc 0.966 f1 0.964 bpcer 0.0 apcer 0.069
0.7 auc 0.948 f1 0.945 bpcer 0.0 apcer 0.103
0.75 auc 0.897 f1 0.885 bpcer 0.0 apcer 0.207
0.8 auc 0.621 f1 0.389 bpcer 0.0 apcer 0.759
0.85 auc 0.5 f1 0.0 bpcer 0.0 apcer 1.0
0.9 auc 0.5 f1 0.0 bpcer 0.0 apcer 1.0
0.95 auc 0.5 f1 0.0 bpcer 0.0 apcer 1.0


In [32]:
# dataframe
data = {'th':ths, 'auc': auc_th, 'bpcer': bpcer_th, 'apcer': apcer_th,
        'tpr': tpr_list, 'fpr': fpr_list}
df_out = pd.DataFrame(data)

# plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_out['apcer'], y=df_out['bpcer'], mode='markers+lines+text', text=df_out['th']))
fig.update_layout(
    title = 'DET',
    xaxis_title = 'apcer',
    yaxis_title = 'bpcer'
)
fig.show()

In [33]:
# plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_out['fpr'], y=df_out['tpr'], mode='markers+lines+text', text=df_out['th']))
fig.update_layout(
    title = 'ROC',
    xaxis_title = 'fpr',
    yaxis_title = 'tpr'
)
fig.show()

Note: Though this implementation achieves a very good performance, it is unusual to see that kind of output. An AUC closer to one is more realistic than a AUC equals to one.